In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
from pdfminer.layout import LTCurve, LTLine, LTTextContainer, LTChar, LTRect
from pdfminer.high_level import extract_text_to_fp
import pandas as pd
import math
import numpy as np

In [ ]:
b = []
for p in extract_pages("all.pdf"):
    b.append(p)

In [ ]:
textboxes = pd.DataFrame(columns=['element','type','leftmargin','verleftline','verrightline','lines','statename','nextsubstate','nextstate'])
lines = pd.DataFrame(columns=['element','tailHorEle','headHorEle','headcurve'])
curves = pd.DataFrame(columns=['element','type','headcurve'])
rects = pd.DataFrame(columns=['element'])
txtbxlist = []
linelist  = []
curvelist = []
rectlist = []
for element in b[40]:
#         print(element)
        if isinstance(element, LTTextContainer):
#             print(element)
            print(element.get_text())
            txtbxlist.append(element)
        elif isinstance(element, LTLine): 
            linelist.append(element)
        elif isinstance(element, LTRect):
            rectlist.append(element)
        elif isinstance(element, LTCurve):
            curvelist.append(element)
textboxes['element'] = txtbxlist
lines['element'] = linelist
curves['element'] = curvelist
rects['element'] = rectlist

In [327]:
for t in textboxes.element:
    print(t)
#     for e in t:
# #         for c in e:
#             print(e.get_text())
#             print(e)

<LTTextBoxHorizontal(0) 15.000,600.551,616.573,607.551 'Printed by Dinesh KURUBA on 12/5/2021 1:32:46 PM. For personal use only. Not approved for distribution. Copyright © 2021 National Comprehensive Cancer Network, Inc., All Rights Reserved.\n'>
<LTTextBoxHorizontal(1) 186.617,542.950,465.716,582.550 'NCCN Guidelines Version 2.2021\nAcute Lymphoblastic Leukemia\n'>
<LTTextBoxHorizontal(2) 651.683,546.291,768.448,583.691 'NCCN Guidelines Index\nTable of Contents\nDiscussion\n'>
<LTTextBoxHorizontal(3) 275.279,505.716,516.718,516.716 'NCCN Categories of Evidence and Consensus\n'>
<LTTextBoxHorizontal(4) 86.937,426.252,671.851,501.272 'Based upon high-level evidence, there is uniform NCCN consensus that the intervention is appropriate.\nCategory 1\nCategory 2A Based upon lower-level evidence, there is uniform NCCN consensus that the intervention is appropriate.\nCategory 2B Based upon lower-level evidence, there is NCCN consensus that the intervention is appropriate.\nCategory 3\nAll rec

In [ ]:
def issamebox(box, ubox):
    for i in range(4):
        if abs(box[i]-ubox[i])>3:
            return False
    return True

def finduniquetable(tables):
#     print(tables)
    unique = [tables[0]]
    for box in tables:
#         print('box={}'.format(box))
        found = False
        for ubox in unique:
#             print('ubox={}'.format(ubox))
            if issamebox(box, ubox):
                found = True
                break
        if not found:
            unique.append(box)
    return unique

def findaxesdistance(pts1,pts2,index):
    out = -1
    if pts1[index] > pts2[index]:
        out = min(pts1[index] - pts2[2+index],pts1[2+index] - pts2[2+index])
    else:
        out = min(pts2[index] - pts1[2+index],pts2[2+index] - pts1[2+index])
    return out if out>0 else 0
    

def findleft(rect, lines):
    rectb = vars(rect.element)['bbox']
    out = -1
    for i,lin in lines.iterrows():
        lbox = vars(lin.element)['bbox']
        height = vars(lin.element)['height']
        width = vars(lin.element)['width']
        if (height > 0) & (lbox[0]<=rectb[0]) & (out<lbox[0]) & (findaxesdistance(rectb,lbox,1)<1):
            out = lbox[0]
    return out

def findbottom(rect, lines):
    rectb = vars(rect.element)['bbox']
    out = -1
    for i,lin in lines.iterrows():
        lbox = vars(lin.element)['bbox']
        height = vars(lin.element)['height']
        width = vars(lin.element)['width']
        if (width > 0) & (lbox[1]<=rectb[1]) & (out<lbox[1]) & (findaxesdistance(rectb,lbox,0)<1):
            out = lbox[1]
    return out

def findright(rect, lines):
    rectb = vars(rect.element)['bbox']
    out = 5000
    for i,lin in lines.iterrows():
        lbox = vars(lin.element)['bbox']
        height = vars(lin.element)['height']
        width = vars(lin.element)['width']
        if (height > 0) & (lbox[0]>=rectb[2]) & (out>lbox[0]) & (findaxesdistance(rectb,lbox,1)<1):
            out = lbox[0]
    return out

def findtop(rect, lines):
    rectb = vars(rect.element)['bbox']
    out = 5000
    for i,lin in lines.iterrows():
        lbox = vars(lin.element)['bbox']
        height = vars(lin.element)['height']
        width = vars(lin.element)['width']
        if (width > 0) & (lbox[1]>=rectb[3]) & (out>lbox[1]) & (findaxesdistance(rectb,lbox,0)<1):
            out = lbox[1]
    return out
    

def findtopbottomline(rect,lines):
    bbox = [-1, -1, 5000, 5000]
    bbox[0] = findleft(rect,lines)
    bbox[1] = findbottom(rect,lines)
    bbox[2] = findright(rect, lines)
    bbox[3] = findtop(rect,lines)
    return bbox


def gettables(rects,lines):
    tables = []
    for i, rect in rects.iterrows():
        rectbbox  = findtopbottomline(rect,lines)
        tables.append(rectbbox)
    return finduniquetable(tables)
    


In [ ]:
tables = gettables(rects,lines)
tables

In [ ]:
def getboundingbox(box,ubox):
#     print(box,ubox)
    return [min(box[0],ubox[0]),min(box[1],ubox[1]),max(box[2],ubox[2]),max(box[3],ubox[3])]

def isinsidebox(box, ubox):
    bbox = getboundingbox(box,ubox)
    return issamebox(bbox, ubox)

def issamecolumn(col1,col2):
#     print(col1,col2)
    for i in range(2):
            if abs(col1[i]-col2[i])>3:
#                 print(col1[i],col2[i])
                return False
    return True

def isuniquecolumn(column,columns):
    if len(columns)==0:
        return True
    
    out = True
    for cl in columns:
        if issamecolumn(cl,column):
            out = False
#             print(out)
            break
#     print(out)
    return out

def findcolumnsfortable(table,rects):
    columns = []
    rows = []
#     print(table)
    height = -1
    for i,rect in rects.iterrows():
            rectbox = vars(rect.element)["bbox"]
            if isinsidebox(rectbox,table):
#                 print('inside',rectbox)
                column = [rectbox[0],rectbox[2]]
                if isuniquecolumn(column,columns):
#                     print('inside',column)
                    columns.append(column)
                    height = rectbox[3] - rectbox[1]
    return columns,height
                

def findcolumnsfortables(tables, rects):
    columns  = []
    rowh = []
    for table in tables:
#         print(table)
        column, height = findcolumnsfortable(table,rects)
        columns.append(column)
        rowh.append(height)
    return columns, rowh

In [ ]:
columns,rowheight = findcolumnsfortables(tables, rects)
columns, rowheight

In [265]:
def getblocktext(bbox,textboxes):
    text = []
    sizes = []
    for e in textboxes.element:
        for l in e:
            for c in l:
                if isinstance(c,LTChar):
                    p = vars(c)
#                     print(p)
                    size = 0
                    if hasattr(c,'size'):
                        size = p['size']
                    cbox = p["bbox"]
                    if isinsidebox(cbox,bbox):
                        ctext = c.get_text()
#                         print(ctext)
#                         print(size)
                        text.append(ctext)
                        sizes.append(size)
#     print(text, sizes)
    return text, sizes


def getrecttabletexts(tables,columns,rowheight,textboxes):
    tablestexts = []
    for i, table in enumerate(tables):
        y0 = table[3]-rowheight[i]
        y1 = table[3]
        tabletexts = []
        while (y0-table[1])>=0:
            rowtxts = []
            for col in columns[i]:
#                 print(col,y0,y1)
                bbox = [col[0],y0,col[1],y1]
                rowtxts.append(getblocktext(bbox,textboxes))
            tabletexts.append(rowtxts)
            y0 = y0-rowheight[i]
            y1 = y1-rowheight[i]
        tablestexts.append(tabletexts)
    return tablestexts

In [270]:
tabtexts = getrecttabletexts(tables,columns,rowheight,textboxes)

In [343]:
def istopbordersharing(box1,box2):
    return (abs(box1[1]-box2[3])<2) & (abs(box1[0]-box2[0])<1) & (abs(box1[2]-box2[2])<1)

def istopbordercontains(box1,box2):
    print(box1, box2)
    return ((box1[1]-box2[3])>0) & ((box1[1]-box2[3])<20) & (box1[0]<=box2[0]) #& (box1[2]>=box2[2])

def gettextLinetext(ele):
    text = []
    sizes = []
    for c in ele:
        if isinstance(c,LTChar):
            p = vars(c)
            size = 0
            if hasattr(c,'size'):
                size = p['size']
            cbox = p["bbox"]
            ctext = c.get_text()
            text.append(ctext)
            sizes.append(size)
#     print(text, sizes)
    return text, sizes

def gettabdesc(box, texts):
    tabdec = ""
    for i,txt in texts.iterrows():
        for line in txt.element:
            p = vars(line)
            if istopbordercontains(box,p['bbox']):
                tabdec = gettextLinetext(line)
                print("tabdec")
                break
    return tabdec

def gettablerel1(tabtexts,textboxes,tables):
    rel1 = []
    rel3 = []
    for i,tab in enumerate(tables):
        for j,tab2 in enumerate(tables):
            if istopbordersharing(tab,tab2):
#                 print(len(tabtexts[j]))
                for row in tabtexts[j]:
                    rel1.append([row[0],tabtexts[i]])
                tabdesc = gettabdesc(tab2,textboxes)
                rel3.append([tabdesc,tabtexts[i]])
    return rel1, rel3

def gettablerel2(tabtexts,textboxes,tables):
    rel2 = []
    for j,tab in enumerate(tables):
        for row in tabtexts[j]:
            if len(row)==2:
                rel2.append([row[0],row[1]])
    return rel2                

def getrecttabrelns(tabtexts,textboxes,tables):
    rel1, rel3 = gettablerel1(tabtexts,textboxes,tables)
    rel2 = gettablerel2(tabtexts,textboxes,tables)
    return rel1, rel2, rel3
    

In [344]:
recttabrelns1, recttabrelns2, recttabrelns3 = getrecttabrelns(tabtexts,textboxes,tables)

[82.94, 441.8185, 709.06, 505.8517] (15.0, 600.551, 616.5730000000004, 607.551)
[82.94, 441.8185, 709.06, 505.8517] (186.6168, 564.5495999999999, 465.71580000000006, 582.5495999999999)
[82.94, 441.8185, 709.06, 505.8517] (186.6168, 542.9495999999999, 456.0210000000001, 560.9495999999999)
[82.94, 441.8185, 709.06, 505.8517] (651.683, 572.6907, 768.437, 583.6907)
[82.94, 441.8185, 709.06, 505.8517] (682.846, 559.4907, 768.448, 570.4907)
[82.94, 441.8185, 709.06, 505.8517] (715.274, 546.2906999999999, 768.448, 557.2906999999999)
[82.94, 441.8185, 709.06, 505.8517] (275.2793, 505.71569999999997, 516.7183000000001, 516.7157)
[82.94, 441.8185, 709.06, 505.8517] (165.64229999999998, 490.2717, 666.3622999999995, 501.2717)
[82.94, 441.8185, 709.06, 505.8517] (86.9373, 489.7107, 143.7853, 500.7107)
[82.94, 441.8185, 709.06, 505.8517] (86.93729999999998, 473.7057, 671.8512999999995, 485.2667)
[82.94, 441.8185, 709.06, 505.8517] (86.93729999999998, 457.7007, 632.1192999999996, 469.2617)
[82.94, 44

In [345]:
len(recttabrelns1), len(recttabrelns2), len(recttabrelns3)
# recttabrelns2[6]

(7, 7, 2)

In [ ]:
with open("relations.csv",'w',encoding='utf-8') as f:
    for e in relations:
        for i in e:
            f.write('"{}",'.format(i).replace('\n','\\n'))
        f.write('\n')